# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

4. Выполнить 3 с помощью "протягиваемых" формул.

## Лабораторная работа 7.1

In [1]:
import pandas as pd
import xlwings as xw
import csv

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [2]:
recipes = pd.read_csv("recipes_sample1.csv")
# Загрузка данных из файла reviews_sample.csv в DataFrame reviews с учётом безымянного столбца индексов
reviews = pd.read_csv("reviews_sample1.csv", index_col=0)


print("recipes:")
print("________")
pd.DataFrame(recipes)[:][:5]

recipes:
________


,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN


In [3]:
print("reviews:")
print("________")
pd.DataFrame(reviews)[:][:5]

reviews:
________


,user_id,recipe_id,date,rating,review
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...


2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [5]:
reviews_sample = reviews.sample(frac=0.05, random_state=42)
recipes_sample = recipes.sample(frac=0.05, random_state=42)

# Запись данных на разные листы в Excel файл
with pd.ExcelWriter('recipes.xlsx') as writer:
    recipes_sample.to_excel(writer, sheet_name='Рецепты', index=False)
    reviews_sample.to_excel(writer, sheet_name='Отзывы', index=False)

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [6]:
# Загрузка данных из файла recipes.xlsx
recipes_df = pd.read_excel('recipes.xlsx', sheet_name='Рецепты')

# Добавление столбца "seconds_assign" (время в секундах)
recipes_df['seconds_assign'] = recipes_df['minutes'] * 60

# Открываем файл recipes.xlsx
wb = xw.Book('recipes.xlsx')
sheet = wb.sheets['Рецепты']

# Присваивание массива значений столбца "seconds_assign" диапазону ячеек в Excel
sheet.range('G1').value = "seconds_assign"
sheet.range('G2').options(index=False, header=False, expand='down').value = recipes_df['seconds_assign']
wb.save()


4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [7]:
recipes_s = pd.read_excel('recipes.xlsx', sheet_name='Рецепты')

wb = xw.Book('recipes.xlsx')
sheet = wb.sheets['Рецепты']

# Устанавливаем диапазон для новой формулы в столбец 
last_row = sheet.range('A1').expand('down').last_cell.row
seconds_formula_range = f'F2:F{last_row}'

# Добавляем заголовок для столбца cекунд
sheet.range('F1').value = "seconds_formula"

# Устанавливаем формулу для столбца "seconds_formula"
sheet.range(seconds_formula_range).formula = "=C2*60" 
wb.save()

5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [10]:
wb = xw.Book('recipes.xlsx')
sheet = wb.sheets['Рецепты']

# Добавление столбца "seconds_formula" на лист "Рецепты" с использованием формул Excel
sheet.range('F1').font.bold = True  # Установка полужирного шрифта
sheet.range('F1').horizontalalignment = -4108  # Выравнивание по центру
sheet.range('G1').font.bold = True  # Установка полужирного шрифта
sheet.range('G1').horizontalalignment = -4108  # Выравнивание по центру

wb.save()

6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [127]:
wb = xw.Book('recipes.xlsx')
sheet = wb.sheets['Рецепты']

cell_all = sheet.range('A:AA')
cell_all.color = None  # Зеленый
wb.save()
wb.close()

In [28]:
# Открываем книгу Excel
wb = xw.Book('recipes.xlsx')
sheet = wb.sheets['Рецепты']

# Получаем доступ к конкретной ячейке
cell = sheet.range('C2:C1503')
for i in cell.value:
    index = cell.value.index(i)
    if int(str(i).split('.')[0]) < 5:
        cell[index].color = (0, 255, 0)
    elif 5 <= int(str(i).split('.')[0]) <= 10:
        cell[index].color = (65535, 65535, 0)
    else:
        cell[index].color = (255, 0, 0)

wb.save()

com_error: (-2147352567, 'Ошибка.', (0, None, None, None, 0, -2146827864), None)

7. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [42]:
wb = xw.Book('recipes.xlsx')
sheet = wb.sheets['Рецепты']
sheet2 = wb.sheets['Отзывы']

# Устанавливаем диапазон для новой формулы в столбец 
last_row = sheet.range('A1').expand('down').last_cell.row
n_reviews_range = f'I2:I{last_row}'

# Добавляем заголовок для столбца отзывов
sheet.range('I1').value = "n_reviews"

# Устанавливаем формулу для столбца "n_reviews"
sheet.range(n_reviews_range).value = "=IFNA(QUERY('Отзывы'!$A1:F1500;"select SUM(D) where B = '"&$B2&"' group by A label SUM(D) ' '";1);"-")"
wb.save()

SyntaxError: invalid syntax (241879486.py, line 13)

In [47]:
wb = xw.Book('recipes.xlsx')
sheet = wb.sheets['Рецепты']
sheet2 = wb.sheets['Отзывы']

sheet.range(n_reviews_range).value = ''
wb.save()

## Лабораторная работа 7.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

9. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука. 

10. При помощи пакета `xlwings` добавьте в столбец J формулу для описания столбца на языке SQL. Формула должна реализовывать следующую логику:

    1\. в начале строки идут значения из столбцов В и C (значение столбца С приведено к верхнему регистру), разделенные пробелом
    
    2\. далее идут слова на основе столбца "Ключ"
        2.1 если в столбце "Ключ" указано значение "PK", то дальше через пробел идет ключевое слово "PRIMARY KEY"
        2.2 если в столбце "Ключ" указано значение "FK", то дальше через пробел идет ключевое слово "REFERENCES", затем значения столбцов H и I в формате "название_таблицы(название_столбца)"
        
    3\. если в столбце "Обязательно к заполнению" указано значение "Y" и в столбце "Ключ" указано не "PK", то дальше через пробел идет ключевое слово "NOT NULL".

Заполните этой формулой необходимое количество строк, используя "протягивание". Количество строк для протягивания определите на основе данных.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

11. При помощи пакета `xlwings` измените стилизацию листа `Модель`.
* для заголовков добавьте заливку цвета `00ccff`
* примените автоподбор ширины столбца;
* сделайте шрифт заголовков полужирным;
* добавьте таблице автофильтр.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

12. Посчитайте количество атрибутов для каждой из сущностей. Создайте лист `Статистика` и запишите в него результат группировки, начиная с ячейки "А1". Визуализируйте полученный результат при помощи столбчатой диаграммы. Сохраните полученную визуализацию на лист `Статистика`, начиная с ячейки "E2".  Сделайте скриншот листа `Статистика` и прикрепите в ячейку ноутбука.

* Вы можете воспользоваться методами для визуализации, которые поставляются вместе с объектами `pandas` (см. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot) 